In [21]:
import pandas as pd
from graphdatascience import GraphDataScience

In [22]:
# neo4j desktop m1, 5.3.0
host = "bolt://localhost:7687"
user = "neo4j"
password= "j4oenj4oen"

gds = GraphDataScience(host, auth=(user, password))
print(gds.version())

2.3.2


In [23]:
# file should be placed in import/
query = """
LOAD CSV FROM $url AS row FIELDTERMINATOR ' '
MERGE (s:Node {id:row[0]})
MERGE (t:Node {id:row[1]})
MERGE (s)-[l:Link]-(t)
SET l.weight = 1
"""
params = {'url': 'file:///11.edges'}
gds.run_cypher(query, params)

""


In [24]:
# in the latest version of neo4j, size() should get a pattern comprehension as parameter, wrong: (c)--(), correct: [p=(c)--() | p]
degree = gds.run_cypher("""
MATCH (n:Node)
RETURN n.id AS nodeId,
       size([p=(n)--() | p]) AS degree
""")

print(degree.head())

  nodeId  degree
0      0       4
1      1       4
2      2       4
3      3       4
4      4       4


In [25]:
query = """
MATCH (n) DETACH DELETE n
"""
gds.run_cypher(query)
gds.close()